In [1]:
#csv파일이랑 audio폴더 test 양만큼 개줄여서 디버그하며 할 것

import os
import librosa
import librosa.display
import IPython.display
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.font_manager as fm
import pandas as pd
import csv

import scipy
from scipy.stats import skew
from tqdm import tqdm, tqdm_pandas

tqdm.pandas()

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

from sklearn.svm import SVC

In [2]:
filename = np.array(pd.read_csv('data/SC_file_name.csv'))
answer = np.array(pd.read_csv('data/SC_answer.csv'))

audio_train_files = os.listdir('./audio_train/')
audio_test_files = os.listdir('./audio_test/')
train=pd.read_csv('./file_name.csv')

In [3]:
#print(audio_train_files)
#print(audio_test_files)

In [4]:
def get_mfcc(name, path):
    data, _ = librosa.core.load(path+name) #엑셀에 .wav까지 붙여 저장해야됨 안그러면 path+name+'.wav' 이렇게해야
    print(data,_)
    #data, _ = librosa.load(path)
    try:
        #ft1 = librosa.feature.mfcc(data, sr = SAMPLE_RATE, n_mfcc=30)
        ft1 = librosa.feature.mfcc(data, n_mfcc=30) #이게핵심 Sample_rate를 어떻게 조절하느냐
        ft2 = librosa.feature.zero_crossing_rate(data)[0] # n_mfcc 값을 어떻게 조절하느냐에 따라 Feature개수가 달리 뽑힘
        ft3 = librosa.feature.spectral_rolloff(data)[0]
        ft4 = librosa.feature.spectral_centroid(data)[0]
        ft5 = librosa.feature.spectral_contrast(data)[0]
        ft6 = librosa.feature.spectral_bandwidth(data)[0]
        ft1_trunc = np.hstack((np.mean(ft1, axis=1), np.std(ft1, axis=1), skew(ft1, axis = 1), np.max(ft1, axis = 1), np.median(ft1, axis = 1), np.min(ft1, axis = 1)))
        ft2_trunc = np.hstack((np.mean(ft2), np.std(ft2), skew(ft2), np.max(ft2), np.median(ft2), np.min(ft2)))
        ft3_trunc = np.hstack((np.mean(ft3), np.std(ft3), skew(ft3), np.max(ft3), np.median(ft3), np.min(ft3)))
        ft4_trunc = np.hstack((np.mean(ft4), np.std(ft4), skew(ft4), np.max(ft4), np.median(ft4), np.min(ft4)))
        ft5_trunc = np.hstack((np.mean(ft5), np.std(ft5), skew(ft5), np.max(ft5), np.median(ft5), np.min(ft5)))
        ft6_trunc = np.hstack((np.mean(ft6), np.std(ft6), skew(ft6), np.max(ft6), np.median(ft6), np.max(ft6)))
        return pd.Series(np.hstack((ft1_trunc, ft2_trunc, ft3_trunc, ft4_trunc, ft5_trunc, ft6_trunc)))
    except:
        print('bad file')
        return pd.Series([0]*210)

In [5]:
train_data = pd.DataFrame()
train_data['fname'] = train['fname']
test_data=pd.DataFrame()
test_data['fname']=audio_test_files

In [6]:
#print(train_data)
print(test_data)


                         fname
0   130_1p3_Al_mc_AKGC417L.wav
1   130_1p3_Ar_mc_AKGC417L.wav
2   130_1p3_Ll_mc_AKGC417L.wav
3   130_1p3_Lr_mc_AKGC417L.wav
4   130_1p3_Pl_mc_AKGC417L.wav
5   130_1p3_Pr_mc_AKGC417L.wav
6   130_1p3_Tc_mc_AKGC417L.wav
7   130_1p4_Al_mc_AKGC417L.wav
8   200_2p4_Al_mc_AKGC417L.wav
9   200_2p4_Ar_mc_AKGC417L.wav
10  200_2p4_Lr_mc_AKGC417L.wav
11  200_2p4_Pl_mc_AKGC417L.wav
12  200_2p4_Pr_mc_AKGC417L.wav
13  200_2p4_Tc_mc_AKGC417L.wav
14  200_3p4_Al_mc_AKGC417L.wav
15  200_3p4_Ar_mc_AKGC417L.wav
16  200_3p4_Pl_mc_AKGC417L.wav
17  200_3p4_Pr_mc_AKGC417L.wav
18  200_3p4_Tc_mc_AKGC417L.wav
19  201_1b1_Al_sc_Meditron.wav
20  201_1b1_Ar_sc_Meditron.wav
21  201_1b2_Al_sc_Meditron.wav
22  201_1b2_Ar_sc_Meditron.wav
23  201_1b3_Al_sc_Meditron.wav
24  201_1b3_Ar_sc_Meditron.wav
25  202_1b1_Ar_sc_Meditron.wav
26  203_1p2_Al_mc_AKGC417L.wav
27  203_1p2_Ar_mc_AKGC417L.wav
28  203_1p2_Lr_mc_AKGC417L.wav
29  203_1p2_Pl_mc_AKGC417L.wav
..                         ...
63  205_

In [7]:
train_data=train_data['fname'].progress_apply(get_mfcc,path='./audio_train/')
print('done loading train mfcc')
test_data = test_data['fname'].progress_apply(get_mfcc, path='./audio_test/')
print('done loading test mfcc')



  0%|                                                                                          | 0/299 [00:00<?, ?it/s]

[0.06973912 0.10266001 0.09116188 ... 0.08305359 0.08018959 0.08612607] 22050


  1%|▌                                                                                 | 2/299 [00:06<15:37,  3.16s/it]

[-0.02710845 -0.03993477 -0.03530764 ... -0.04230448 -0.0414599
 -0.04527015] 22050


  1%|▊                                                                                 | 3/299 [00:09<14:54,  3.02s/it]

...

 99%|█████████████████████████████████████████████████████████████████████████████████ | 92/93 [04:37<00:02,  2.69s/it]

[ 0.          0.          0.         ... -0.22099693 -0.21257466
 -0.23776534] 22050


100%|██████████████████████████████████████████████████████████████████████████████████| 93/93 [04:39<00:00,  2.68s/it]

[ 0.          0.          0.         ... -0.05103435 -0.05238643
 -0.05545169] 22050


done loading test mfcc


In [8]:
train_data['fname'] = train['fname']
train_data['label'] = train['label']

test_data['fname'] = audio_test_files
test_data['label'] = np.zeros((len(audio_test_files)))

In [9]:
#print(train_data)
print(test_data)

             0           1          2          3          4          5  \
0  -374.568476  121.653345  68.564955  35.947216  30.422790  31.903702   
1  -397.516973   97.320405  71.506663  47.443845  33.989614  28.778871   
2  -309.153916  108.590946  49.665790  39.516736  30.495293  28.091219   
3  -270.489556  161.700804  49.739469  17.809513  13.739956  18.044573   
4  -266.609507  158.940668  69.044580  38.555959  28.130570  21.142545   
5  -286.585566  149.852568  74.347056  30.249630  16.780876  14.380454   
6  -243.604437  144.016071   9.083950  25.720157  23.012074  19.829217   
7  -333.424073  134.835047  62.990353  28.851085  28.611614  31.429581   
8  -267.313418   96.330247  20.233180  28.502039  25.393421  21.782721   
9  -222.562185  136.710127  38.600022  30.539879  30.992133  32.360694   
10 -244.793261  110.656752  43.300727  37.123352  24.064541  25.433031   
11 -251.776179  112.736151  52.861586  47.129276  28.793808  27.698803   
12 -281.348908   99.808704  44.638440 

[93 rows x 212 columns]


In [10]:
X = train_data.drop(['label', 'fname'], axis=1)
feature_names = list(X.columns)
X = X.values
labels = np.sort(np.unique(train_data.label.values))
num_class = len(labels)
c2i = {}
i2c = {}
for i, c in enumerate(labels):
    c2i[c] = i
    i2c[i] = c
y = np.array([c2i[x] for x in train_data.label.values])

#print(X)
#print(len(X)) - 99
#y는 train 데이터들이 라벨링한 것
#print(y)
#print(len(y)) - 99

In [11]:
X_test = test_data.drop(['label', 'fname'], axis=1)
X_test = X_test.values

#print(X_test)
#print(len(X_test)) - 8

In [12]:
print(X.shape)
print(X_test.shape)
#test의 라벨링은 어디서?

(299, 210)
(93, 210)


In [13]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_test_scaled = scaler.transform(X_test)
#이놈이 뭘까

In [14]:
#pca = PCA(n_components=65).fit(X_scaled)
#65땜에 오류뜸
pca = PCA(n_components=65).fit(X_scaled)
#이 값 고쳐야
X_pca = pca.transform(X_scaled)
X_test_pca = pca.transform(X_test_scaled)

print(sum(pca.explained_variance_ratio_)) 

0.9667856951166022


In [15]:
# Fit an SVM model
X_train, X_val, y_train, y_val = train_test_split(X_pca, y, test_size = 0.2, random_state = 42, shuffle = True)
#test_size와 random_state 확인

#train 관련
#print(X_train)
#print(y_train) - train 라벨링

#train 내에서 쪼개는 것으로 추측됨 test 0.2가 100개중 20개로 맞네
#test를 따로 만드는게 아니라 train 내에서 일부를 test에 사용하여 정확도 측정
print(X_val)
print(y_val)
#print(len(X_val)) - 20
#print(len(y_val)) - 20

clf = SVC(kernel = 'rbf', probability=True)

clf.fit(X_train, y_train)

print(accuracy_score(clf.predict(X_val), y_val))

[[-6.0574237   5.53631483 -1.14552842 ... -0.19398018  0.50498533
  -0.38345381]
 [-2.68798566  1.18116591 -3.64737395 ... -0.05154531 -0.107796
   0.25035027]
 [ 3.6630428   6.59873133  3.17546579 ... -0.26343876 -0.23017746
  -0.42441702]
 ...
 [-1.23513999  3.80868895 -1.28549982 ... -0.53509076  0.20488332
   0.36786328]
 [-3.86487187 -7.87938391 -2.05387047 ... -0.67389615  0.05712335
  -0.3960214 ]
 [ 0.48513732 -8.87678328 -4.73662835 ...  0.63959184  0.29561464
  -0.31895134]]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


0.9833333333333333


In [16]:
C_grid = [0.001, 0.01, 0.1, 1, 10]
gamma_grid = [0.001, 0.01, 0.1, 1, 10]
param_grid = {'C': C_grid, 'gamma' : gamma_grid}

#grid = GridSearchCV(SVC(kernel='rbf'), param_grid, cv = 3, scoring = "accuracy")
#이부분 수정해야
#데이터 갯수 확 느니까 바로먹음
grid = GridSearchCV(SVC(kernel='rbf'), param_grid, cv = 3, scoring = "accuracy")
grid.fit(X_train, y_train)

# Find the best model
print(grid.best_score_)

print(grid.best_params_)

print(grid.best_estimator_)

0.9665271966527197
{'C': 10, 'gamma': 0.01}
SVC(C=10, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=0.01, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)


In [18]:
clf = SVC(kernel = 'rbf', C = 10, gamma = 0.01, probability=True)

clf.fit(X_train, y_train)

print(accuracy_score(clf.predict(X_val), y_val))

0.9833333333333333
